In [1]:
import numpy as np
import sys
import pandas as pd
sys.path.insert(0, "/Users/mallen2/OpenPATH_Data/e-mission-server")

import emission.core.get_database as edb
Stage_analysis_timeseries = edb.get_analysis_timeseries_db()
Error_Rates = edb.get_Error_Rates_db()

import emission.storage.timeseries.aggregate_timeseries as estag 
agts = estag.AggregateTimeSeries()   

inferred_trips_query = Stage_analysis_timeseries.find({
    "$and": [ {"metadata.key":"analysis/confirmed_trip"},
            {"data.algorithm_chosen": {"$in": ["label_assist","sensing"]}}]
    }) 

inferred_trips_df = agts.to_data_df("analysis/confirmed_trip",inferred_trips_query)

LABEL_CATEGORIES = ['mode_confirm','purpose_confirm','replaced_mode']

intensity_df = pd.read_csv("/Users/mallen2/OpenPATH_Data/em-public-dashboard/viz_scripts/auxiliary_files/energy_intensity.csv")

storage not configured, falling back to sample, default configuration
Connecting to database URL localhost


OperationFailure: An existing index has the same name as the requested index. When index names are not specified, they are auto generated and can cause conflicts. Please refer to our documentation. Requested index: { v: 2, key: { data.start_loc: "2dsphere" }, name: "data.start_loc_2dsphere", sparse: true, 2dsphereIndexVersion: 3 }, existing index: { v: 2, key: { data.start_loc: "2dsphere" }, name: "data.start_loc_2dsphere", 2dsphereIndexVersion: 3 }, full error: {'ok': 0.0, 'errmsg': 'An existing index has the same name as the requested index. When index names are not specified, they are auto generated and can cause conflicts. Please refer to our documentation. Requested index: { v: 2, key: { data.start_loc: "2dsphere" }, name: "data.start_loc_2dsphere", sparse: true, 2dsphereIndexVersion: 3 }, existing index: { v: 2, key: { data.start_loc: "2dsphere" }, name: "data.start_loc_2dsphere", 2dsphereIndexVersion: 3 }', 'code': 86, 'codeName': 'IndexKeySpecsConflict'}

In [ ]:
def make_rel_errors(max_size):
    upper= np.random.rand(1)*max_size
    lower = -np.random.rand(1)*max_size
    return  [float(lower),float(upper)]

Error_Rates = edb.get_Error_Rates_db()
# mode count intervals, (purpose count intervals later), carbon intervals, energy intervals
# Maybe eventually provide the bootstrap distributions?
modes = ['Walk', 'Regular Bike', 'Pilot ebike', 'Scooter share', 'Car, drove alone', 'Car, with others', 'Taxi/Uber/Lyft', 'Bus', 'Train', 'Free Shuttle', 'Air', 'Other', 'Not a Trip','Total']
mode_rel_errors = {}
carbon_rel_errors = {}
energy_rel_errors = {}
for m in modes:
    if m is not 'Total':
        mode_rel_errors[m] = make_rel_errors(0.15)
    carbon_rel_errors[m] = make_rel_errors(0.2)
    energy_rel_errors[m] = make_rel_errors(0.2)

mode_rel_errors['Not a Trip'] = make_rel_errors(0.15)

error_dictionary = {}
error_dictionary["counts"] = {}
error_dictionary["counts"]["mode_confirm"] = mode_rel_errors
error_dictionary["carbon"] = carbon_rel_errors
error_dictionary["energy"] = energy_rel_errors
error_dictionary["distance"] = {"interval": make_rel_errors(0.05), "mean": 0.01, "variance":0.02}
Error_Rates.drop()
Error_Rates.insert_one(error_dictionary)

# to reset: 

In [15]:
# Run this once. If you want to change it, drop the collection and rerun the above cell.
Error_Rates.drop()

In [11]:
Error_Rates.find_one()['counts']['mode_confirm']

{'Walk': [0.02014541220218981, 0.0627340422390293],
 'Regular Bike': [0.12004246462890271, 0.11140137636098851],
 'Pilot ebike': [0.0552292621236988, 0.0677333882442257],
 'Scooter share': [0.1376255330351625, 0.048115242394656324],
 'Car, drove alone': [0.06808967932281694, 0.04556396594562888],
 'Car, with others': [0.03509083218756289, 0.021046977946456318],
 'Taxi/Uber/Lyft': [0.059927213901637306, 0.0278107051151504],
 'Bus': [0.11130283709735216, 0.023026379603005798],
 'Train': [0.039355328102518135, 0.04979931021805307],
 'Free Shuttle': [0.0058088717624887095, 0.1459446432478618],
 'Air': [0.0452573612197445, 0.03190513630895882],
 'Other': [0.028378039916156506, 0.06297062246444944],
 'Not a Trip': [0.03097256302532974, 0.10706292622151638]}

### Compute Metrics with Error Bars
The following functions compute metrics and error bars using inferred trip labels. These can be added to the values calculated from trips that have user input.

In [ ]:
import count_functions as counts

db_errors = Error_Rates.find_one()
rel_count_errors = db_errors["counts"]
all_inferred_counts = counts.get_inferred_counts(inferred_trips_df, LABEL_CATEGORIES)
count_intervals = counts.get_count_intervals(all_inferred_counts,LABEL_CATEGORIES, rel_count_errors)

In [ ]:
inferred_trips_df['sensed_mode']

In [ ]:
import footprint_metrics as fm

carbon_rel_errors = db_errors["carbon"]
energy_rel_errors = db_errors["energy"]
distances_by_mode = fm.get_distances_by_mode(inferred_trips_df)
carbon_by_mode = fm.get_inferred_carbon_intervals(distances_by_mode, carbon_rel_errors,intensity_df)
energy_by_mode = fm.get_inferred_energy_intervals(distances_by_mode, energy_rel_errors,intensity_df)

# are the distances in meters in the database?

In [ ]:
energy_by_mode

In [ ]:
distances_by_mode

In [ ]:
intensity_df #[intensity_df["mode"] == "Car, drove alone"]["CO2_factor"]